In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import json
import os
import random
import glob
# from detectron_pro import detectron_mask_img,detectron_mask_img_composite
import shutil
import cv2

import random
from tqdm import tqdm

In [2]:

# to get location that stickimg will sticked on jpg_dir center or random
def center_location(stickimg_dir):
    #im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    x_center,y_center = im.shape[1]/2,im.shape[0]/2
    #im_stick = np.array(Image.open(stickimg_dir), dtype=np.uint8)
    #im_stick_shape = im_stick.shape
    im_stick_shape = (128, 64)
    bd_box_x,bd_box_y = x_center-(im_stick_shape[1]/2),y_center-(im_stick_shape[0]/2)
    bd_box_length,bd_box_height =im_stick_shape[1],im_stick_shape[0]
    
    result = [[bd_box_x,bd_box_y,bd_box_length,bd_box_height]]
    return result

def random_location(num = None):
    #im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    # x boundary
    #rangeX = 400 
    #x_left_bound,x_right_bound = rangeX, im.shape[1]-rangeX
    result = []
    
    seq = [[320, [120, 240]], [280, [80, 160]], [240, [64, 128]], [220, [54, 108]], [200, [48, 96]], [190, [36, 72]], [180, [30, 60]], [175, [28, 56]]]
    
    if num == None:
        re = random.sample(seq, 1)
    else:
        re = random.sample(seq, len(num))
    
    x_left_bound,x_right_bound = 128,512
    
    for i in range(len(re)):
        x_center = random.randint(x_left_bound, x_right_bound)
        result.append([x_center-int(re[i][1][0]/2), re[i][0]-int(re[i][1][1]/2), re[i][1][0], re[i][1][1], num[i]])
    
    return result


#h_list = [200,220,240,296,352] # from top to buttom (center of people image)
#h_scale = [(48,96),(54,108),(64,128),(80,160),(120,240)] # top might be smaller

In [3]:
#it can generate json file defined by your street_jpg and people_jpg 

def create_json_file(jpg_dir,street_json,results_dir,function='center',num_of_people = 1):
    if function == 'center':
        re = center_location(stickimg_dir)
    elif function == 'random':
        re = random_location(num = num_of_people)
    
        
    input_file = open (street_json)
    json_array = json.load(input_file)
    
    data = []
    for box in re:
        data.append({
        'end':0,
        'hide':0,
        'id':0,
        'init':0,
        'lbl':"pasted_person",
        'lock':0,
        'occl':0,
        'pos':[
        box[0],
        box[1],
        box[2],
        box[3]],    
        'posv':[
        0,
        0,
        0,
        0],
        'str':0,
        'img':box[4]
        })
    
    if json_array != []:
        for item in json_array:
            data.append(item)
            
    with open(results_dir, 'w') as outfile:
        json.dump(data, outfile)

# Start create

In [4]:
origin_data_dir = '/root/notebooks/final/caltech_pedestrian_refine_no'

In [5]:
img_data = glob.glob(origin_data_dir+'/street/*.jpg', recursive=True)

json_data = glob.glob(origin_data_dir+'/street_json/*.json', recursive=True)

In [6]:
img_data[:10]

['/root/notebooks/final/caltech_pedestrian_refine_no/street/014786.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/034453.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/052326.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/052361.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/052380.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/052518.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/071931.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/071956.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/091533.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/093033.jpg']

In [7]:
json_data[:10]

['/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013258.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013259.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013260.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013261.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013262.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013263.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013264.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013265.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013266.json',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street_json/013267.json']

In [8]:
# Image read dir
street_dir = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/video_extractor/*'

people_dir = '/root/notebooks/final/market_mask2'

# Image save dir
save_dir = '/root/notebooks/final/result_dataset_13'

num_imgs = 42000

In [9]:
# Check dir folder exit
# If not, create one
if os.path.exists(save_dir) == False:
    os.makedirs(save_dir)

for s in ['people', 'mask', 'street', 'street_json','json']:
    if os.path.exists(os.path.join(save_dir, s)) == False:
        os.makedirs(os.path.join(save_dir, s))

In [10]:
#street_imgs = glob.glob(street_dir+'/**/*.jpg', recursive=True)
street_imgs = img_data

#street_imgs = random.shuffle(random.sample(street_imgs, 5000))
street_imgs = random.sample(street_imgs, num_imgs)

random.shuffle(street_imgs)

In [11]:
#people_imgs_ = glob.glob(people_dir+'/people/*.jpg', recursive=True)

mask_imgs_ = glob.glob(people_dir+'/market_mask_refine_6467/*.jpg', recursive=True)

people_imgs_ = [i.replace('market_mask_refine_6467', 'people') for i in mask_imgs_]

#mask_imgs_ = [i.replace('people', 'mask') for i in people_imgs_]

#people_imgs = random.sample(people_imgs, num_imgs)

r_t_imgs = []
r_m_imgs = []

In [12]:
mask_imgs_[:10]

['/root/notebooks/final/market_mask2/market_mask_refine_6467/003301.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/013112.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/001516.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/011705.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/011063.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/001270.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/002779.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/013674.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/013660.jpg',
 '/root/notebooks/final/market_mask2/market_mask_refine_6467/010369.jpg']

In [13]:
# for i in range(len(people_imgs_)):
#     if (i%1000==0):
#         print("Process (",i,"/",len(people_imgs_),")  ","{:.2f}".format(100*i/len(people_imgs_))," %")
#     r_t_imgs.append(cv2.imread(people_imgs_[i]))
#     r_m_imgs.append(cv2.imread(mask_imgs_[i]))

In [14]:
#people_imgs = [people_imgs_[random.randint(0,len(people_imgs_)-1)] for i in range(num_imgs)]


#random.shuffle(people_imgs)

In [15]:
street_imgs[:10]

['/root/notebooks/final/caltech_pedestrian_refine_no/street/062504.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/014390.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/026876.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/068185.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/009531.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/024540.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/074649.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/040897.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/066308.jpg',
 '/root/notebooks/final/caltech_pedestrian_refine_no/street/028768.jpg']

In [16]:
# copy people image and mask to dir

pbar = tqdm(total=len(people_imgs_))

for i in range(len(people_imgs_)):
    shutil.copyfile(mask_imgs_[i], save_dir+'/mask/'+str('{0:06}'.format(i))+'.jpg')
    shutil.copyfile(people_imgs_[i], save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg')
    pbar.update()
    
pbar.close()

100%|██████████| 6466/6466 [01:48<00:00, 59.50it/s]


In [17]:
for i in range(num_imgs):
    
    if (i%100==0):
        print("Process (",i,"/",num_imgs,")  ","{:.2f}".format(100*i/num_imgs)," %")
        
    people_pick = random.randint(0,len(people_imgs_)-1)
    
    # create mask and save
    #try:
    #    mask_img = detectron_mask_img(people_imgs[i],(64,128))
    #    mask_img = Image.fromarray(mask_img)
    #except Exception as e:
    #    print("Skip image :",i)
    #    continue
    
    #mask_img = mask_imgs_[people_pick]
        
    #mask_img.save(save_dir+'/mask/'+str('{0:06}'.format(i))+'.jpg')
    
    #shutil.copyfile(mask_img, save_dir+'/mask/'+str('{0:06}'.format(i))+'.jpg')
    
    # save street img
    street_img = cv2.imread(street_imgs[i])
    street_img = cv2.resize(street_img,(640,480))
    cv2.imwrite(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg', street_img)
    
    ################################################################
    img_path = street_imgs[i]
    json_dir = img_path.replace('street', 'street_json')
    json_dir = json_dir.replace('jpg', 'json')
    shutil.copyfile(json_dir, save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json')
    ################################################################
    
    # save poeple img
    #people_img = cv2.imread(people_imgs[i])
    people_img = people_imgs_[people_pick]
    #people_img = cv2.resize(people_img,(64,128))
    #cv2.imwrite(save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg', people_img)
    #shutil.copyfile(people_img, save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg')
    
    if random.randint(0,100)>50:
        nump = [str('{0:06}'.format(random.randint(0,len(people_imgs_)-1)))+'.jpg' for i in range(2)]
    else:
        nump = [str('{0:06}'.format(random.randint(0,len(people_imgs_)-1)))+'.jpg' for i in range(3)]
    
    # create json file and save
    create_json_file(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg',
                     save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json',
                     save_dir+'/json/'+str('{0:06}'.format(i))+'.json',
                     function="random",
                     num_of_people = nump)

Process ( 0 / 42000 )   0.00  %
Process ( 100 / 42000 )   0.24  %
Process ( 200 / 42000 )   0.48  %
Process ( 300 / 42000 )   0.71  %
Process ( 400 / 42000 )   0.95  %
Process ( 500 / 42000 )   1.19  %
Process ( 600 / 42000 )   1.43  %
Process ( 700 / 42000 )   1.67  %
Process ( 800 / 42000 )   1.90  %
Process ( 900 / 42000 )   2.14  %
Process ( 1000 / 42000 )   2.38  %
Process ( 1100 / 42000 )   2.62  %
Process ( 1200 / 42000 )   2.86  %
Process ( 1300 / 42000 )   3.10  %
Process ( 1400 / 42000 )   3.33  %
Process ( 1500 / 42000 )   3.57  %
Process ( 1600 / 42000 )   3.81  %
Process ( 1700 / 42000 )   4.05  %
Process ( 1800 / 42000 )   4.29  %
Process ( 1900 / 42000 )   4.52  %
Process ( 2000 / 42000 )   4.76  %
Process ( 2100 / 42000 )   5.00  %
Process ( 2200 / 42000 )   5.24  %
Process ( 2300 / 42000 )   5.48  %
Process ( 2400 / 42000 )   5.71  %
Process ( 2500 / 42000 )   5.95  %
Process ( 2600 / 42000 )   6.19  %
Process ( 2700 / 42000 )   6.43  %
Process ( 2800 / 42000 )   6.67 